#BabyAGI User Guide

В этом блокноте демонстрируется реализация [BabyAGI](https://github.com/yoheinakajima/babyagi/tree/main) Йохея Накаджимы.

BabyAGI - это рекурсивный агент искусственного интеллекта, который может генерировать и претендовать, что выполняет задания на основе заданной цели.

Хотя BabyAGI использует конкретные векторные хранилища/провайдеры моделей (Pinecone, OpenAI), одним из преимуществ реализации с помощью LangChain является то, что их можно легко заменить на другие варианты.

В данной реализации мы используем векторное хранилище FAISS (поскольку оно работает локально и является бесплатным).

# Установка и импорт необходимых модулей

In [9]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


In [2]:
!pip install langchain_experimental


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [11]:
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.1 MB/s eta 0:00:00


In [16]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00


In [3]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain_experimental.autonomous_agents import BabyAGI

[LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html)

[OpenAI](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html)

[PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html)

[OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html)

[BaseLLM](https://api.python.langchain.com/en/latest/llms/langchain.llms.base.BaseLLM.html)

[VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.base.VectorStore.html)

[Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.base.Chain.html)

#Connect to the Vector Store

В зависимости от того, какой векторный магазин вы используете, этот шаг может выглядеть по-разному.



In [4]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

API Reference:

[FAISS](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html)

[InMemoryDocstore](https://api.python.langchain.com/en/latest/docstore/langchain.docstore.in_memory.InMemoryDocstore.html)

In [6]:
import ipywidgets as widgets
from IPython.display import display
import os

def on_button_click(b):
    os.environ['OPENAI_API_KEY'] = key_input.value
    print("Ключ OpenAI_API установлен!")

key_input = widgets.Password(
    placeholder='Введите ваш ключ API здесь',
    description='API ключ:',
    disabled=False
)
apply_button = widgets.Button(description="Применить")
apply_button.on_click(on_button_click)
display(key_input, apply_button)

Password(description='API ключ:', placeholder='Введите ваш ключ API здесь')

Button(description='Применить', style=ButtonStyle())

Ключ OpenAI_API установлен!


In [7]:
openai_api_key = key_input.value
os.environ['OPENAI_API_KEY'] = openai_api_key

In [12]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

#Run the BabyAGI
Теперь пришло время создать контроллер BabyAGI и посмотреть, как он пытается выполнить поставленную задачу.

In [13]:
OBJECTIVE = "Напиши прогноз погоды в Москве на сегодня"
llm = OpenAI(temperature=0)

In [14]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [19]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

7: Write a forecast for the chance of thunderstorms in Moscow for today.
8: Write a forecast for the chance of precipitation in Moscow for today.
9: Gather data on the current temperature in Moscow.
10: Gather data on the current humidity in Moscow.
11: Analyze the data to make a prediction about the humidity in Moscow for the next 48 hours.
12: Create a visual representation of the expected temperature in Moscow for the next 48 hours.
13: Create a visual representation of the expected humidity in Moscow for the next 48 hours.
14: Analyze the data to make a prediction about the wind direction in Moscow for the next 48 hours.
15: Create a timeline of the expected wind speed in Moscow for the next 48 hours.
16: Analyze the data to make a prediction about the temperature in Moscow for the next 48 hours.
17: Analyze the data to make a prediction about the chance of thunderstorms in Moscow for the next 48 hours.
18: Analyze the data to make a prediction about the chanc

{'objective': 'Напиши прогноз погоды в Москве на сегодня'}

    
    *****TASK LIST*****
    
    1: Make a todo list
    
    *****NEXT TASK*****
    
    1: Make a todo list
    
    *****TASK RESULT*****
    
    
    
    1. Check the weather forecast for San Francisco today
    2. Make note of the temperature, humidity, wind speed, and other relevant weather conditions
    3. Write a weather report summarizing the forecast
    4. Check for any weather alerts or warnings
    5. Share the report with the relevant stakeholders
    
    *****TASK LIST*****
    
    2: Check the current temperature in San Francisco
    3: Check the current humidity in San Francisco
    4: Check the current wind speed in San Francisco
    5: Check for any weather alerts or warnings in San Francisco
    6: Check the forecast for the next 24 hours in San Francisco
    7: Check the forecast for the next 48 hours in San Francisco
    8: Check the forecast for the next 72 hours in San Francisco
    9: Check the forecast for the next week in San Francisco
    10: Check the forecast for the next month in San Francisco
    11: Check the forecast for the next 3 months in San Francisco
    1: Write a weather report for SF today
    
    *****NEXT TASK*****
    
    2: Check the current temperature in San Francisco
    
    *****TASK RESULT*****
    
    
    
    I will check the current temperature in San Francisco. I will use an online weather service to get the most up-to-date information.
    
    *****TASK LIST*****
    
    3: Check the current UV index in San Francisco.
    4: Check the current air quality in San Francisco.
    5: Check the current precipitation levels in San Francisco.
    6: Check the current cloud cover in San Francisco.
    7: Check the current barometric pressure in San Francisco.
    8: Check the current dew point in San Francisco.
    9: Check the current wind direction in San Francisco.
    10: Check the current humidity levels in San Francisco.
    1: Check the current temperature in San Francisco to the average temperature for this time of year.
    2: Check the current visibility in San Francisco.
    11: Write a weather report for SF today.
    
    *****NEXT TASK*****
    
    3: Check the current UV index in San Francisco.
    
    *****TASK RESULT*****
    
    
    
    The current UV index in San Francisco is moderate. The UV index is expected to remain at moderate levels throughout the day. It is recommended to wear sunscreen and protective clothing when outdoors.
    
    *****TASK ENDING*****
    





    {'objective': 'Write a weather report for SF today'}